In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
OSCILLATOR = '153'  # 153, 154 or 155
TIME_FRAME = 'M'  # 'H', 'M' or 'S'
EXPORTED_TIMESTAMP = '1656423962'

def get_file_path(oscillator, time_frame, dataset):
    if time_frame == 'H':
        return f"./datasets/10.15.237.{oscillator}_hr_{EXPORTED_TIMESTAMP}/{dataset}.csv"
    elif time_frame == 'M':
        return f"./datasets/10.15.237.{oscillator}_min_{EXPORTED_TIMESTAMP}/{dataset}.csv"
    elif time_frame == 'S':
        return f"./datasets/10.15.237.{oscillator}_sec_{EXPORTED_TIMESTAMP}/{dataset}.csv"

## Delete Constant State and Process Lock State

Queremos conseguir los Timestamps donde el oscilador esté en estado Lock estable; es decir, que el valor de xoState sea $4$

In [ ]:
def get_raw_states(oscillator, time_frame):
    states_path = get_file_path(oscillator,time_frame,'xoState')
    
    df = pd.read_csv(states_path)
    df = df.drop(['End: -1'], axis=1)
    df.columns = ['Timestamp','State']
    df = df.drop(0, axis=0)
    df['State'] = df['State'].astype(np.float64)
    
    return df

In [ ]:
df = get_raw_states(OSCILLATOR,TIME_FRAME)

In [ ]:
plt.figure(figsize=(12,6))
plt.title("State through time")
sns.set_style("darkgrid")
sns.lineplot(data=df["State"], color="#c6474b")
#plt.ylabel("State")
sns.despine()
#plt.savefig("fotos/state_through_time_constant_lock_included.png")

In [ ]:
def delete_constant_lock_values(df):
    for i in range(len(df)):
        if df.iloc[len(df)-i-1]['State'] != 4.0:
            constant_lock_start = df.iloc[len(df)-i]['Timestamp']
            break
    
    df = df[:df[df['Timestamp']==constant_lock_start].index.values[0]]
    return df

In [ ]:
df = delete_constant_lock_values(df)

In [ ]:
plt.figure(figsize=(16,7))
plt.subplot(1, 2, 1)
plt.title("Original")
sns.set_style("darkgrid")
sns.lineplot(data=df["State"], color="#c6474b")
#plt.ylabel("State")
sns.despine()

#plot 2:
plt.subplot(1, 2, 2)
plt.title("Zoom")
sns.set_style("darkgrid")
sns.lineplot(data=df["State"],color="#c6474b")
plt.ylabel("")
sns.despine()
plt.xlim([175,190])
plt.suptitle("Clock State Through Time")
plt.tight_layout()
#plt.savefig("fotos/state_through_time.png")

In [ ]:
len(df)

In [ ]:
def get_lock_state_timestamps(raw_df):
    df = raw_df.loc[raw_df["State"]==4]
    df = df.drop("State",axis=1)
    
    return df

In [ ]:
df = get_lock_state_timestamps(df)

In [ ]:
len(df)

In [ ]:
def join_features(df, oscillator, time_frame):
    xoDAC = get_file_path(oscillator,time_frame,'xoDac_xoState-Lock')

    dac_df = pd.read_csv(xoDAC)
    dac_df = dac_df.drop(['Start: 0','End: -1'], axis=1)
    dac_df.columns = ['Timestamp','DAC']
    dac_df = dac_df.drop(0, axis=0)
    df = df.merge(dac_df,on='Timestamp')
    
    
    xoPhaseError = get_file_path(oscillator,time_frame,'xoPhaseError_xoState-Lock')
    
    phase_df = pd.read_csv(xoPhaseError)
    phase_df = phase_df.drop(['Start: 0','End: -1'], axis=1)
    phase_df.columns = ['Timestamp','Phase Error']
    phase_df = phase_df.drop(0, axis=0)
    df = df.merge(phase_df,on='Timestamp') 
    
    
    sysOscTemp = get_file_path(oscillator,time_frame,'sysOscTemp')
    
    temp_df = pd.read_csv(sysOscTemp)
    temp_df = temp_df.drop('End: -1', axis=1)
    temp_df.columns = ['Timestamp','Temperature']
    temp_df = temp_df.drop(0, axis=0)
    df = df.merge(temp_df,on='Timestamp')
    
    
    xoFreqError = get_file_path(oscillator,time_frame,'xoFreqError_xoState-Lock')
    
    freq_df = pd.read_csv(xoFreqError)
    freq_df = freq_df.drop(['Start: 0','End: -1'], axis=1)
    freq_df.columns = ['Timestamp','Frequency Error']
    freq_df = freq_df.drop(0, axis=0)
    df = df.merge(freq_df,on='Timestamp')
    
    return df

In [ ]:
def format_features(df):
    df[['DAC','Phase Error','Temperature','Frequency Error']] = df[['DAC','Phase Error','Temperature','Frequency Error']].astype(np.float64)
    df['Timestamp'] = df['Timestamp'].astype(int)
    return df

In [ ]:
df = join_features(df, OSCILLATOR, TIME_FRAME)
df = format_features(df)

In [ ]:
print('Number of missing values:')
print(df.isna().sum())

In [ ]:
df.head()

In [ ]:
df.to_csv("datasets/processed_data.csv", index=False)

In [ ]:
plt.figure(figsize=(16,7))
#plt.subplot(1, 2, 1)
plt.title("DAC (Target)")
sns.set_style("darkgrid")
sns.lineplot(data=df["DAC"].values, color="#c6474b")
#plt.ylabel("State")
sns.despine()
plt.xlim([0,3584])

#plt.savefig("fotos/raw_DAC_value_to_predict.png")

## Split DAC data into stages

In [ ]:
df = get_raw_states(OSCILLATOR,TIME_FRAME)
df = delete_constant_lock_values(df)
df = get_lock_state_timestamps(df)
df = join_features(df, OSCILLATOR, TIME_FRAME)
df = format_features(df)

In [ ]:
def get_state_change_by_timestamp(oscillator, time_frame):
    state_info = get_file_path(oscillator,time_frame,'xoState_change-true')
    aux_df = pd.read_csv(state_info)
    aux_df = aux_df.drop(['Start: 0','End: -1'], axis=1)
    aux_df.columns = ['Timestamp','Lock=4']
    aux_df = aux_df.drop(0, axis=0)
    aux_df['Lock=4'] = aux_df['Lock=4'].astype(np.float64)
    
    return aux_df

In [ ]:
state_info = get_state_change_by_timestamp(OSCILLATOR, TIME_FRAME)

In [ ]:
state_info.head(10)

In [ ]:
def get_stages_info(df,state_info,var):
    stages = np.array([])
    new_stage = np.array([])
    start_stage = False
    init_timestamp = int(0)
    count = 0
    for index,row in state_info.iterrows():
        if start_stage == True:
            for i,r in df.iterrows():
                if int(r['Timestamp']) >= init_timestamp and int(r['Timestamp']) <= int(row['Timestamp']):
                    new_stage = np.append(new_stage,r[var])
                    #print(new_stage)
                    
            stages = np.append(stages,[count,new_stage])
            count += 1
            #print(stages)
            new_stage = np.array([])
            start_stage = False
        if row['Lock=4']==4:
            init_timestamp = int(row['Timestamp'])
            start_stage=True
    
    stages = np.delete(stages,34)
    stages = np.delete(stages,34)
    stages = stages.reshape(int(len(stages)/2),2)

    return stages

In [ ]:
dac_stages = get_stages_info(df,state_info,"DAC")
freq_stages = get_stages_info(df,state_info,"Frequency Error")
ph_stages = get_stages_info(df,state_info,"Phase Error")
temp_stages = get_stages_info(df,state_info,"Temperature")

In [ ]:
plt.figure(figsize=(12,6))
plt.title('Etapas de disciplinado (sin preprocesado)')
for stage in dac_stages:
    sns.lineplot(data=stage[1])

sns.set_style("darkgrid")
plt.xlabel('Timestamp')
plt.ylabel('DAC')
sns.despine()

#plt.savefig('./fotos/etapas_sin_preprocesado.png')

In [ ]:
def stabilize_stages(m,stages):
    for stage in stages:
        if len(stage[1]) < m:
            m=len(stage[1])

    for stage in stages:
        while len(stage[1]) > m:
            stage[1] = np.delete(stage[1],0)
    
    return stages

In [ ]:
dac_stages = stabilize_stages(150,dac_stages)
freq_stages = stabilize_stages(150,freq_stages)
ph_stages = stabilize_stages(150,ph_stages)
temp_stages = stabilize_stages(150,temp_stages)

In [ ]:
plt.figure(figsize=(14,12))
plt.suptitle('          Etapas estables de disciplinado',fontsize=18)
plt.subplot(4, 1, 1)

plt.title('DAC')
for stage in dac_stages:
    sns.lineplot(data=stage[1])

sns.set_style("darkgrid")

sns.despine()

plt.subplot(4, 1, 2)

plt.title('Error de Frecuencia')
for stage in freq_stages:
    sns.lineplot(data=stage[1])

sns.set_style("darkgrid")

sns.despine()

plt.subplot(4, 1, 3)

plt.title('Error de Fase')
for stage in ph_stages:
    sns.lineplot(data=stage[1])

sns.set_style("darkgrid")

sns.despine()

plt.subplot(4, 1, 4)

plt.title('Temperatura')
for stage in temp_stages:
    sns.lineplot(data=stage[1])

sns.set_style("darkgrid")

sns.despine()

plt.tight_layout()

#plt.savefig("fotos/etapas_juntas_todas_variables153.png")

## Join Features according to stages

In [ ]:
concat_dac_stages = np.array([])
for stage in dac_stages:
    concat_dac_stages = np.append(concat_dac_stages,stage[1])
concat_freq_stages = np.array([])
for stage in freq_stages:
    concat_freq_stages = np.append(concat_freq_stages,stage[1])
concat_ph_stages = np.array([])
for stage in ph_stages:
    concat_ph_stages = np.append(concat_ph_stages,stage[1])
concat_temp_stages = np.array([])
for stage in temp_stages:
    concat_temp_stages = np.append(concat_temp_stages,stage[1])

df = pd.DataFrame()
df['DAC'] = concat_dac_stages
df['Frequency Error'] = concat_freq_stages
df['Phase Error'] = concat_ph_stages
df['Temperature'] = concat_temp_stages

In [ ]:
df.head()

In [ ]:
plt.figure(figsize=(16,7))
#plt.subplot(1, 2, 1)
plt.title("DAC")
sns.set_style("darkgrid")
sns.lineplot(data=df["DAC"].values, color="#c6474b")
#plt.ylabel("State")
sns.despine()
#plt.xlim([0,3584])
#plt.savefig('./fotos/DAC_n_valores_fuera.png')

In [ ]:
df.to_csv("datasets/processed_data_with_stages.csv")

## Check Second-rate data

In [ ]:
EXPORTED_TIMESTAMP = "1656423428"
TIME_FRAME = "S"
df = get_raw_states(OSCILLATOR,TIME_FRAME)
#df = delete_constant_lock_values(df)
df = get_lock_state_timestamps(df)
df = join_features(df, OSCILLATOR, TIME_FRAME)
df = format_features(df)

In [ ]:
plt.figure(figsize=(16,7))
#plt.subplot(1, 2, 1)
plt.title("DAC medido en Segundos")
sns.set_style("darkgrid")
sns.lineplot(data=df["DAC"].values, color="#c6474b")
#plt.ylabel("State")
sns.despine()
plt.xlim([0,6749])

plt.savefig("fotos/secondrateDAC.png")

## Process All Oscilators Data

### Oscilator 154

In [ ]:
OSCILLATOR="154"
EXPORTED_TIMESTAMP="1656424100"
TIME_FRAME="M"
df1 = get_raw_states(OSCILLATOR,TIME_FRAME)

In [ ]:
plt.figure(figsize=(12,6))
plt.title("State through time")
sns.set_style("darkgrid")
sns.lineplot(data=df1["State"], color="#c6474b")
#plt.ylabel("State")
sns.despine()

In [ ]:
df1 = get_lock_state_timestamps(df1)
df1 = join_features(df1, OSCILLATOR, TIME_FRAME)
df1 = format_features(df1)

In [ ]:
plt.figure(figsize=(16,7))
#plt.subplot(1, 2, 1)
plt.title("DAC (Target)")
sns.set_style("darkgrid")
sns.lineplot(data=df1["DAC"].values, color="#c6474b")
#plt.ylabel("State")
sns.despine()
plt.xlim([0,3584])

In [ ]:
state_info = get_state_change_by_timestamp(OSCILLATOR, TIME_FRAME)

In [ ]:
dac_stages1 = get_stages_info(df1,state_info,"DAC")
freq_stages1 = get_stages_info(df1,state_info,"Frequency Error")
ph_stages1 = get_stages_info(df1,state_info,"Phase Error")
temp_stages1 = get_stages_info(df1,state_info,"Temperature")

In [ ]:
dac_stages1 = stabilize_stages(150,dac_stages1)
freq_stages1 = stabilize_stages(150,freq_stages1)
ph_stages1 = stabilize_stages(150,ph_stages1)
temp_stages1 = stabilize_stages(150,temp_stages1)

In [ ]:
plt.figure(figsize=(14,12))
plt.suptitle('          Etapas estables de disciplinado (Reloj=154)',fontsize=18)
plt.subplot(4, 1, 1)

plt.title('DAC')
for stage in dac_stages1:
    sns.lineplot(data=stage[1])

sns.set_style("darkgrid")

sns.despine()

plt.subplot(4, 1, 2)

plt.title('Error de Frecuencia')
for stage in freq_stages1:
    sns.lineplot(data=stage[1])

sns.set_style("darkgrid")

sns.despine()

plt.subplot(4, 1, 3)

plt.title('Error de Fase')
for stage in ph_stages1:
    sns.lineplot(data=stage[1])

sns.set_style("darkgrid")

sns.despine()

plt.subplot(4, 1, 4)

plt.title('Temperatura')
for stage in temp_stages1:
    sns.lineplot(data=stage[1])

sns.set_style("darkgrid")

sns.despine()

plt.tight_layout()

#plt.savefig("fotos/etapas_juntas_todas_variables154.png")

In [ ]:
concat_dac_stages = np.array([])
for stage in dac_stages1:
    concat_dac_stages = np.append(concat_dac_stages,stage[1])
concat_freq_stages = np.array([])
for stage in freq_stages1:
    concat_freq_stages = np.append(concat_freq_stages,stage[1])
concat_ph_stages = np.array([])
for stage in ph_stages1:
    concat_ph_stages = np.append(concat_ph_stages,stage[1])
concat_temp_stages = np.array([])
for stage in temp_stages1:
    concat_temp_stages = np.append(concat_temp_stages,stage[1])

df1 = pd.DataFrame()
df1['DAC'] = concat_dac_stages
df1['Frequency Error'] = concat_freq_stages
df1['Phase Error'] = concat_ph_stages
df1['Temperature'] = concat_temp_stages
df = df.append(df1, ignore_index=True)


In [ ]:
len(df)

In [ ]:
df1.to_csv("datasets/processed_data_with_stages_154.csv")

### Oscillator 155

In [ ]:
OSCILLATOR="155"
EXPORTED_TIMESTAMP="1656424222"
TIME_FRAME="M"
df2 = get_raw_states(OSCILLATOR,TIME_FRAME)

In [ ]:
plt.figure(figsize=(12,6))
plt.title("State through time")
sns.set_style("darkgrid")
sns.lineplot(data=df2["State"], color="#c6474b")
#plt.ylabel("State")
sns.despine()
#plt.xlim([600,1000])

In [ ]:
df2 = delete_constant_lock_values(df2)
df2 = get_lock_state_timestamps(df2)
df2 = join_features(df2, OSCILLATOR, TIME_FRAME)
df2 = format_features(df2)

In [ ]:
plt.figure(figsize=(16,7))
#plt.subplot(1, 2, 1)
plt.title("DAC (Target)")
sns.set_style("darkgrid")
sns.lineplot(data=df2["DAC"].values, color="#c6474b")
#plt.ylabel("State")
sns.despine()
#plt.xlim([0,3584])

In [ ]:
state_info = get_state_change_by_timestamp(OSCILLATOR, TIME_FRAME)
while state_info["Lock=4"].value_counts()[4] != state_info["Lock=4"].value_counts()[3]:
    state_info = state_info.drop(state_info[state_info['Lock=4'].eq(3).shift(1, fill_value=False) & state_info['Lock=4'].eq(4).shift(-1, fill_value=False) & state_info['Lock=4'].eq(4)].index, axis=0)

In [ ]:
dac_stages2 = get_stages_info(df2,state_info,"DAC")
freq_stages2 = get_stages_info(df2,state_info,"Frequency Error")
ph_stages2 = get_stages_info(df2,state_info,"Phase Error")
temp_stages2 = get_stages_info(df2,state_info,"Temperature")

In [ ]:
dac_stages2 = stabilize_stages(150,dac_stages2)
freq_stages2 = stabilize_stages(150,freq_stages2)
ph_stages2 = stabilize_stages(150,ph_stages2)
temp_stages2 = stabilize_stages(150,temp_stages2)

In [ ]:
plt.figure(figsize=(14,12))
plt.suptitle('          Etapas estables de disciplinado (Reloj=155)',fontsize=18)
plt.subplot(4, 1, 1)

plt.title('DAC')
for stage in dac_stages2:
    sns.lineplot(data=stage[1])

sns.set_style("darkgrid")

sns.despine()

plt.subplot(4, 1, 2)

plt.title('Error de Frecuencia')
for stage in freq_stages2:
    sns.lineplot(data=stage[1])

sns.set_style("darkgrid")

sns.despine()

plt.subplot(4, 1, 3)

plt.title('Error de Fase')
for stage in ph_stages2:
    sns.lineplot(data=stage[1])

sns.set_style("darkgrid")

sns.despine()

plt.subplot(4, 1, 4)

plt.title('Temperatura')
for stage in temp_stages2:
    sns.lineplot(data=stage[1])

sns.set_style("darkgrid")

sns.despine()

plt.tight_layout()

#plt.savefig("fotos/etapas_juntas_todas_variables155.png")

In [ ]:
concat_dac_stages = np.array([])
for stage in dac_stages2:
    concat_dac_stages = np.append(concat_dac_stages,stage[1])
concat_freq_stages = np.array([])
for stage in freq_stages2:
    concat_freq_stages = np.append(concat_freq_stages,stage[1])
concat_ph_stages = np.array([])
for stage in ph_stages2:
    concat_ph_stages = np.append(concat_ph_stages,stage[1])
concat_temp_stages = np.array([])
for stage in temp_stages2:
    concat_temp_stages = np.append(concat_temp_stages,stage[1])

df2 = pd.DataFrame()
df2['DAC'] = concat_dac_stages
df2['Frequency Error'] = concat_freq_stages
df2['Phase Error'] = concat_ph_stages
df2['Temperature'] = concat_temp_stages
df = df.append(df2, ignore_index=True)

In [ ]:
len(df)

In [ ]:
df2.to_csv("datasets/processed_data_with_stages_155.csv")
df.to_csv("datasets/processed_data_with_stages_all_oscillators.csv")